# Co-enhance session 3: 画像収集

## 環境設定 (Google drive内のファイルへのアクセス)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os

# チーム番号 (必ず自分のチームにする！)
os.environ['TEAM'] = ''
print('you are team {}'.format(os.environ['TEAM']))

## Flickr APIを利用した画像収集

In [0]:
# flickrapiモジュールのインストール
# importするために必要
!pip3 install flickrapi

In [0]:
# flickrのapiモジュール
from flickrapi import FlickrAPI
# ファイルのダウンロードモジュール
from urllib.request import urlretrieve
# その他のpython標準モジュール
import time
import sys

In [0]:
# keyとsecretはflickr API登録時に取得したものを入力
key = ""
secret = ""
# 画像ダウンロード間に置く時間 (APIによるサーバ負荷を軽減するため)
waittime = 1
# データの保存先ディレクトリの設定
team = os.getenv('TEAM')
root_dir = "./gdrive/Team Drives/coenhance_teams/team{}/content/data".format(team)

In [0]:
# キーワード(keyword)を含む画像をダウンロードして指定した名前(pth)のディレクトリに保存
def download_images(keyword, pth):
    savedir = os.path.join(root_dir, pth)
    if not os.path.exists(savedir):
        os.mkdir(savedir)
    # FlickrAPIの利用
    flickr = FlickrAPI(key, secret, format="parsed-json")
    # 画像のダウンロード
    res = flickr.photos.search(
        text = keyword,
        # ダウンロードする枚数の設定 (1000枚で試すことを推奨)
        per_page = 1000,
        media = "photos",
        sort = "relevance",
        safe_search = 1,
        extras = "url_q, license")
    photos = res["photos"]
    try:
        for i, photo in enumerate(photos["photo"]):
            # 画像のURLの取得
            url_q = photo["url_q"]
            # xxxxxxxx.jpgという名前にする
            filename = photo["id"] + ".jpg"
            fullpath = os.path.join(savedir, filename)
            # fullpathというパスが存在したら (画像がすでにダウンロードされていたら)無視して次の画像に移る
            if os.path.exists(fullpath):
                continue
            print("{}枚目のダウンロード，URL: {}".format(i+1, url_q))
            # url_qというURLからファイルをダウンロードし、fullpathというパスに保存する
            urlretrieve(url_q, fullpath)
            time.sleep(waittime)
    except:
        import traceback
        traceback.print_exc()

In [0]:
# 実行のための関数 (さっきのdownload_images関数をまとめている！)
def main():
    download_images("マグロ寿司", "sushi")
    download_images("サラダ", "salad")
    download_images("麻婆豆腐", "tofu")
    print("done!")

In [0]:
# 実際にmain関数をここで実行する！
main()

## ダウンロードしたデータの可視化

In [0]:
# 画像ファイル読み込みモジュールPIL
from PIL import Image
# 行列モジュールnumpy
import numpy as np
# 可視化・グラフ作成モジュールmatplotlib
import matplotlib.pyplot as plt
import random

In [0]:
# sushiディレクトリのパス
tardir = os.path.join(root_dir, "sushi")

# sushifilesという画像へのパスを含んだリストを作成
sushifiles = os.listdir(tardir)
sushifiles = [os.path.join(tardir, path) for path in sushifiles]
print(sushifiles[:5])

In [0]:
# キャンバスの設定 (幅, 高さ)
fig = plt.figure(figsize=(20, 10))
# 見せる画像の枚数
show_size = 10

# sushifilesリストをシャッフルする
random.shuffle(sushifiles)

# 画像枚数分繰り返す
for i in range(show_size):
    im = Image.open(sushifiles[i])
    im_arr = np.asarray(im, "uint8")
    fig.add_subplot(1, show_size, i+1)
    plt.imshow(im_arr)
    plt.grid(False)
    plt.axis('off')
plt.show()